In [ ]:
from ai import ai_mtdf
from marubatsu import Marubatsu

mb = Marubatsu()
for f in range(-2, 4):
    print(f"f = {f}")
    ai_mtdf(mb, f=f, shortest_victory=True, use_tt=True,  calc_score=True,
            calc_count=True, debug=True)
    print()

f = -2
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  171/  171 |  (-3, -2) |    -2.0 | fail high | [-2.0,  inf]
  171/  342 |  (-2.0, -1.0) |    -1.0 | fail high | [-1.0,  inf]
  206/  548 |  (-1.0, 0.0) |     0 | fail high | [   0,  inf]
  520/ 1068 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = -1
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  261/  261 |  (-2, -1) |    -1.0 | fail high | [-1.0,  inf]
  206/  467 |  (-1.0, 0.0) |     0 | fail high | [   0,  inf]
  520/  987 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = 0
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  356/  356 |  (-1,  0) |     0 | fail high | [   0,  inf]
  521/  877 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = 1
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  700/  700 |  ( 0,  1) |     0 | fail low  | [-inf,    0]
  118/  818 |  (-1,  0) |     0 | fail high | [   0,    0]

f = 2
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  661/  661 |  ( 1,  2) |    1.0 | fail low 

In [2]:
from ai import ai_by_score, dprint
from copy import deepcopy

@ai_by_score
def ai_mtdf(mb, debug=False, shortest_victory=False, init_ab=False, use_tt=False, 
            f=0, ai_for_mo=None, params={}, sort_allnodes=False, calc_count=False):           
    count = 0
    def ab_search(mborig, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        count += 1
        if mborig.status == Marubatsu.CIRCLE:
            return (11 - mborig.move_count) / 2 if shortest_victory else 1
        elif mborig.status == Marubatsu.CROSS:
            return (mborig.move_count - 10) / 2 if shortest_victory else -1
        elif mborig.status == Marubatsu.DRAW:
            return 0
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if ai_for_mo is not None:
            if sort_allnodes:
                score_by_move = ai_for_mo(mborig, analyze=True, **params)["score_by_move"]
                score_by_move_list = sorted(score_by_move.items(), key=lambda x:x[1], reverse=True)
                legal_moves = [x[0] for x in score_by_move_list]
            else:
                legal_moves = mborig.calc_legal_moves()
                bestmove = ai_for_mo(mborig, rand=False, **params)
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = max(score, ab_search(mb, tt, alpha, beta))
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                score = min(score, ab_search(mb, tt, alpha, beta))
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt in boardtxtlist:
                tt[boardtxt] = (lower_bound, upper_bound)
        return score
                
    min_score = -2 if shortest_victory else -1
    max_score = 3 if shortest_victory else 1
    lbound = min_score if init_ab else float("-inf")
    ubound = max_score if init_ab else float("inf")

    tt = {}
    dprint(debug, "      count | ウィンドウ | αβ 値 |      type |    MM の範囲")
    while lbound != ubound:
        beta = f + 1 if lbound == f else f
        prevcount = count
        f = ab_search(mb, tt, alpha=beta - 1, beta=beta)
        if f >= beta:
            lbound = f
            type = "fail high"
        else:
            ubound = f
            type = "fail low "
        dprint(debug, f"{count - prevcount:5.0f}/{count:5.0f} |  ({beta - 1:2.0f}, {beta:2.0f}) |    {f:2.0f} | {type} | [{lbound:4.0f}, {ubound:4.0f}]")
 
    score = f
            
    if calc_count:
        return count
    if mb.turn == Marubatsu.CIRCLE:
        score *= -1
    return score

In [ ]:
for f in range(-2, 4):
    print(f"f = {f}")
    ai_mtdf(mb, f=f, shortest_victory=True, use_tt=True,  calc_score=True, 
            calc_count=True, debug=True)
    print()

f = -2
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  171/  171 |  (-3, -2) |    -2 | fail high | [  -2,  inf]
  171/  342 |  (-2, -1) |    -1 | fail high | [  -1,  inf]
  206/  548 |  (-1,  0) |     0 | fail high | [   0,  inf]
  520/ 1068 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = -1
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  261/  261 |  (-2, -1) |    -1 | fail high | [  -1,  inf]
  206/  467 |  (-1,  0) |     0 | fail high | [   0,  inf]
  520/  987 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = 0
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  356/  356 |  (-1,  0) |     0 | fail high | [   0,  inf]
  521/  877 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = 1
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  700/  700 |  ( 0,  1) |     0 | fail low  | [-inf,    0]
  118/  818 |  (-1,  0) |     0 | fail high | [   0,    0]

f = 2
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  661/  661 |  ( 1,  2) |     1 | fail low  | [-inf,    1]
 

In [4]:
mb.move(1, 1)
mb.move(0, 0)
for f in range(-2, 4):
    print(f"f = {f}")
    ai_mtdf(mb, f=f, shortest_victory=True, use_tt=True,  calc_score=True, calc_count=True, debug=True)
    print()

f = -2
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
   71/   71 |  (-3, -2) |    -2 | fail high | [  -2,  inf]
   30/  101 |  (-2, -1) |    -1 | fail high | [  -1,  inf]
   58/  159 |  (-1,  0) |     0 | fail high | [   0,  inf]
  136/  295 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = -1
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
   83/   83 |  (-2, -1) |    -1 | fail high | [  -1,  inf]
   58/  141 |  (-1,  0) |     0 | fail high | [   0,  inf]
  136/  277 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = 0
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
   99/   99 |  (-1,  0) |     0 | fail high | [   0,  inf]
  136/  235 |  ( 0,  1) |     0 | fail low  | [   0,    0]

f = 1
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  174/  174 |  ( 0,  1) |     0 | fail low  | [-inf,    0]
   47/  221 |  (-1,  0) |     0 | fail high | [   0,    0]

f = 2
      count | ウィンドウ | αβ 値 |      type |    MM の範囲
  172/  172 |  ( 1,  2) |     0 | fail low  | [-inf,    0]
 

In [5]:
from util import load_mblist, load_bestmoves
from ai import ai10s, ai14s, ai_gt6, ai_scout

bestmoves_by_board = load_bestmoves("../data/bestmoves_by_board.dat")
gt6_params = {"bestmoves_by_board": bestmoves_by_board}
mblist = load_mblist("../data/mblist_by_board_min2.dat")
for ai_for_mo, params in [(ai10s, {}), (ai14s, {}), (ai_gt6, gt6_params)]:
    for shortest_victory in [False, True]:
        print(f"ai_for_mo: {ai_for_mo} shortest_victory: {shortest_victory}")
        print("      | scout |  mtdf | mtdf(mo)")
        count = 0
        count2 = 0
        count3 = 0
        for mb in mblist:
            count += ai_scout(mb, ai_for_mo=ai_for_mo, params=params, 
                                shortest_victory=shortest_victory, use_tt=True,
                                calc_score=True, calc_count=True)
            x, y = ai_for_mo(mb, **params)
            mbcopy = deepcopy(mb)
            mbcopy.move(x, y)
            f = ai_scout(mbcopy, shortest_victory=shortest_victory, use_tt=True, calc_score=True)
            count2 += ai_mtdf(mb, shortest_victory=shortest_victory, f=f, 
                              use_tt=True, calc_score=True, calc_count=True)
            count3 += ai_mtdf(mb, shortest_victory=shortest_victory, f=f, 
                              ai_for_mo=ai_for_mo, params=params, use_tt=True,
                              calc_score=True, calc_count=True)
        
        print(f"count | {count:5} | {count2:5} |    {count3:5}")
        print(f"ratio |       |  {count2 / count * 100:3.0f}% |     {count3 / count * 100:3.0f}%")

ai_for_mo: <function ai10s at 0x00000213B5BE51C0> shortest_victory: False
      | scout |  mtdf | mtdf(mo)
count | 13562 | 20420 |    11188
ratio |       |  151% |      82%
ai_for_mo: <function ai10s at 0x00000213B5BE51C0> shortest_victory: True
      | scout |  mtdf | mtdf(mo)
count | 14508 | 25239 |    15115
ratio |       |  174% |     104%
ai_for_mo: <function ai14s at 0x00000213B5BE56C0> shortest_victory: False
      | scout |  mtdf | mtdf(mo)
count | 11609 | 19727 |     8323
ratio |       |  170% |      72%
ai_for_mo: <function ai14s at 0x00000213B5BE56C0> shortest_victory: True
      | scout |  mtdf | mtdf(mo)
count | 11698 | 24275 |    11974
ratio |       |  208% |     102%
ai_for_mo: <function ai_gt6 at 0x00000213B5BE5E40> shortest_victory: False
      | scout |  mtdf | mtdf(mo)
count | 13383 | 19644 |     9947
ratio |       |  147% |      74%
ai_for_mo: <function ai_gt6 at 0x00000213B5BE5E40> shortest_victory: True
      | scout |  mtdf | mtdf(mo)
count | 14419 | 24471 |    15